In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

'''
    Author:	 Matthew Ogden and Graham West
    Created:	01 Sep 2020
Description:	This code is a modified version of code written by Graham West at https://github.com/gtw2i/GA-Galaxy. 
                The pupose of this code is to genetrically modify the SPAM orbital parameters for galactic models.
'''

# Add parent directory
from sys import path as sysPath
from os import listdir
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Genetic_Algorithm import main_Genetic_Algorithm as ga
import main_SIMR as simr
sm = simr.sm

# General modules
from os import path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# Print testing
gm.test()
im.test()
ga.test()
simr.test()

def test():
    print("SM: Hi!  You're in Matthew's main code for all things simulation.")

# For testing and developement from outside. 
new_func = None
def set_new_func( new_func_ptr ):
    global new_func
    new_func = new_func_ptr
# End set new function
    

GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
GA: Hi!  You're in Matthew's main code for all things genetric algorithm.
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [3]:
tDir = '../targetDir'
print('Before: \'%s\' \nAfter: \'%s\'' % (tDir,gm.validPath(tDir) ))
tDir = gm.validPath(tDir)
rInfo = None

tInfo = im.target_info_class( targetDir = tDir, printAll=True)
if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")
    im.tabprint("Grabbing first model.")
    rInfo = tInfo.getRunInfo()
    #print(rInfo)
    if type(rInfo) == type(None):
        print("BAD RUN!!!!")
    else:
        print("RUN GOOOD!!!")
        im.tabprint('rInfo status: %s' % (rInfo.status) )

Before: '../targetDir' 
After: '/home/mbo2d/587722984435351614/'
IM: target_info_class.__init__:
	 - targetDir:  /home/mbo2d/587722984435351614/
	 - arg.targetDir:  None
IM: Target.initTargetDir():
	 - targetDir: /home/mbo2d/587722984435351614/
	 - targetDir: (True) /home/mbo2d/587722984435351614/
	 - infoDir: (True) /home/mbo2d/587722984435351614/information/
	 - baseInfoLoc: (True) /home/mbo2d/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /home/mbo2d/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /home/mbo2d/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /home/mbo2d/587722984435351614/plots/
IM: Target: Opening target info json
Target Good!
	 - Grabbing first model.
RUN GOOOD!!!
	 - rInfo status: True


# TO-DO List
- Ask about boundary files
- Ask about Maxes and Mins Graham's Code uses from Zoo Merger Files. 
- Ask about the trailing values on the Galaxy Zoo Merger Models files and is it ok to cut them like Graham does? 

In [4]:
zScores, mData = tInfo.getOrbParam()
print(zScores)
print(mData)

tParam = mData[0,:]  # target orbital parameters

[0.94594595 0.94117647 0.91891892 ... 0.         0.         0.        ]
[[-9.93853 -4.5805   3.27377 ...  0.       0.       0.     ]
 [-9.93853 -4.5805  20.84411 ...  1.       0.       0.     ]
 [-9.93853 -4.5805  18.86315 ...  0.       0.       0.     ]
 ...
 [-9.93853 -4.5805  18.17628 ...  1.       0.       0.     ]
 [-9.93853 -4.5805  -7.327   ...  0.       0.       0.     ]
 [-9.93853 -4.5805   0.86396 ...  0.       0.       0.     ]]


In [5]:

pFile = '../Genetic_Algorithm/587722984435351614_combined.txt'
print(path.exists(pFile))

zooThresh = 0.5

import math
from copy import deepcopy
data, psi, nModel, nCol = ga.ReadAndCleanupData( pFile, zooThresh )
print("data: ", data.shape)
print("PSI: ", psi.shape)
print("nModel: ", nModel)
print("nCol: ", nCol)


True
Cleaning target file...
data:  (121, 15)
PSI:  (121, 2)
nModel:  121
nCol:  15


In [6]:
ga_param = {}
ga_param['limits'] = {}
names = sm.spam_param_names
print(names)

['sec_vec_1', 'sec_vec_2', 'sec_vec_3', 'sec_vec_4', 'sec_vec_5', 'sec_vec_6', 'mass_1', 'mass_2', 'rout_1', 'rout_2', 'phi_1', 'phi_2', 'theta_1', 'theta_2', 'epsilon_1', 'epsilon_2', 'rscale_1_1', 'rscale_1_2', 'rscale_1_3', 'rscale_2_1', 'rscale_2_2', 'rscale_2_3']


In [7]:
#####################
###   VARIABLES   ###
#####################


# target ID
zooBase = "587722984435351614"
targetInd = 0
nParam    = 14		# number of SPAM parameters
shrink   = 1.0		# fraction of max xLim

pReal   = data[targetInd,0:-1]
psiReal = psi[targetInd,:]
nBin = 200

# read max xLim and window bounds
XXX = np.loadtxt( "../Genetic_Algorithm/Bounds_587722984435351614.txt" )
#	mmm = XXX[:nParam,:]
bound = XXX[nParam:,:]

# get parameter stats
mins = np.min( data, axis=0 )[0:-1]
maxs = np.max( data, axis=0 )[0:-1]
mmm = [ mins, maxs ]
mmm = np.transpose(np.array(mmm))
#	print mmm
# modify max xLim
#	mmm[2,:]  = np.array([ -10.0,  10.0 ])
#	mmm[2,:]  = np.array([   0.0,   5.0 ])
mmm[2,:]  = np.array([   0.0,   mmm[2,1] ])  # sym
"""
mmm[3,:]  = np.array([  -0.9,   0.9 ])
mmm[4,:]  = np.array([   0.0, 360.0 ])
mmm[5,:]  = np.array([ -90.0,  90.0 ])
#	mmm[6,:]  = np.array([   0.2,   0.8 ])
#	mmm[7,:]  = np.array([  10.0,  70.0 ])
#	mmm[8,:]  = np.array([   0.5,  10.0 ])
#	mmm[9,:]  = np.array([   0.5,  10.0 ])
"""
mmm[10,:] = np.array([   0.0, 180.0 ])	# sym
mmm[11,:] = np.array([   0.0, 180.0 ])	# sym
mmm[12,:] = np.array([ -89.0,  89.0 ])	
mmm[13,:] = np.array([ -89.0,  89.0 ])	
# shrink xLim
xLim = np.zeros((nParam,2))
for i in range(nParam):
    xLim[i,0] = pReal[i] - shrink*(pReal[i] - mmm[i,0])
    xLim[i,1] = shrink*(mmm[i,1] - pReal[i]) + pReal[i]
# end

print("    min      real     max")
for i in range(nParam):
    print( "{:8.2f} ".format(xLim[i,0]) + "{:8.2f} ".format(pReal[i]) + "{:8.2f} ".format(xLim[i,1]) )
    ga_param['limits'][names[i]] = {}
    ga_param['limits'][names[i]]['min'] = xLim[i,0]
    ga_param['limits'][names[i]]['max'] = xLim[i,1]
# end
print(" ")
gm.pprint(ga_param)

    min      real     max
   -9.94    -9.94    -9.94 
   -4.58    -4.58    -4.58 
    0.00     3.27    24.90 
   -0.39     0.29     0.98 
  146.41   258.49   283.50 
  -67.00   -23.28    76.38 
    0.30     0.49     0.80 
   18.18    48.02    65.98 
    2.38     9.13    18.57 
    3.34    17.65    33.76 
    0.00   129.69   180.00 
    0.00    36.81   180.00 
  -89.00   -41.78    89.00 
  -89.00    51.43    89.00 
 
{'limits': {'mass_1': {'max': 0.7971159219741821,
                       'min': 0.30287885665893555},
            'mass_2': {'max': 65.98039245605469,
                       'min': 18.178529739379883},
            'phi_1': {'max': 180.0,
                      'min': 0.0},
            'phi_2': {'max': 180.0,
                      'min': 0.0},
            'rout_1': {'max': 18.574907302856445,
                       'min': 2.3761796951293945},
            'rout_2': {'max': 33.7622184753418,
                       'min': 3.3420610427856445},
            'sec_vec_1': {'max': -9.

In [8]:
gpLoc = '../param/ga_test_param.json'
listdir('../param/')
#gm.saveJson( ga_param, gpLoc )
labels = [ 'x', 'y', 'z', 'vx', 'vy', 'vz', 'mf', 'mt', 'rp', 'rs', 'pp', 'ps', 'tp', 'ts']
print(labels)

['x', 'y', 'z', 'vx', 'vy', 'vz', 'mf', 'mt', 'rp', 'rs', 'pp', 'ps', 'tp', 'ts']


In [9]:
gaLoc = '../Genetic_Algorithm/'
listdir( gaLoc )

# Add custom modules
sysPath.append(gaLoc)
import working_ga as spam_ga

#solve( param, nParam, nBin, bound, fileInd, psi )
# T, V, RVt, RVv = solve( pReal, nParam, nBin, bound, "00", psiReal )
pCall = spam_ga.solve( pReal, nParam, nBin, bound, "00", psiReal )
print('done')

../Genetic_Algorithm/basic_run_unpreturbed -o 00 -9.93853,-4.5805,3.27377,-0.500079,-2.4556503,-1.0779899,23.33004,24.694267,3.49825,5.32056,129.69229,36.8125,-41.7847,51.42857,0 > SolveMetro.out
RV.shape (2001, 6)
done


### Graham's Code and Model Data
This is how Graham's GA code reads in the Galazy Zoo: Merger files and converts them to his GA coordinate system.
It also converts it back into the SPAM coordinate system before calling SPAM. 
NOTE:  His code converts back into one of the 8 true symmetries he identifies in his dissertation. 

In [10]:
print(" i :   Model 1  :   pReal  :  paramStr  : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, tParam[i], pReal[i], pCall[i], tParam[i]-pCall[i]) )

 i :   Model 1  :   pReal  :  paramStr  : diff


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:


gaData, gaPsi= ga.convert_spam_to_ga(mData)
p2 = gaData[0,:]
p2_psi = gaPsi[0,:]

print(pReal.shape)
print(p2.shape)
print(p2_psi)

print(" i :    real   : converted :   target  : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, pReal[i], p2[i], tParam[i], pReal[i]-p2[i]) )

In [ ]:
def convert_ga_to_spam( in_param, psi ):
    
    # Expand in parameters if a single vector
    if len( in_param.shape ) == 1:
        in_param = np.expand_dims(in_param, axis=0)
    
    # Create seperate matrix for output parameters
    out_param = deepcopy(in_param)
    
    for i in range( out_param.shape[0]):
        
        # add for psi
        if( psi[i,0] == -1 ):
            out_param[i,12] += 180.0
        # end
        if( psi[i,1] == -1 ):
            out_param[i,13] += 180.0
        # end

        # convert mass units
        f    = out_param[i,6]
        t    = out_param[i,7]
        out_param[i,6] = f*out_param[i,7]
        out_param[i,7] = (1.0-f)*out_param[i,7]

        G = 1
        c = out_param[i,3]
        v = ( (1+c)/(1-c)*2*G*out_param[i,6]/(out_param[i,0]**2+out_param[i,1]**2+out_param[i,2]**2)**0.5 )**0.5    

        vx = v*math.cos(out_param[i,4]*np.pi/180.0)*math.cos(out_param[i,5]*np.pi/180.0)
        vy = v*math.sin(out_param[i,4]*np.pi/180.0)*math.cos(out_param[i,5]*np.pi/180.0)
        vz = v*math.sin(out_param[i,5]*np.pi/180.0)
        out_param[i,3] = vx
        out_param[i,4] = vy
        out_param[i,5] = vz
        
        Ap = out_param[i,8]
        As = out_param[i,9]
        out_param[i,8] = np.abs(Ap/np.cos(out_param[i,12]*np.pi/180.0))**0.5
        out_param[i,9] = np.abs(As/np.cos(out_param[i,13]*np.pi/180.0))**0.5
        
    return out_param

print(p2_psi)

spamData = ga.convert_ga_to_spam( gaData, gaPsi )
p3 = spamData[0,:]

#print(tParam)
print("tParam",tParam.shape)
print("p2", p2.shape)
print("p3", p3.shape)

print(" i :    target   :   p2   :     p3    : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, p2[i], p3[i], tParam[i], tParam[i]-p3[i]) )


## Comparison
Here we can see my code matches Graham's Code for the very first model.  This was not tested on any other model parameters. 

In [ ]:

print(" i :   target  :   pCall   :     p3    : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, tParam[i], pCall[i], p3[i], pCall[i]-p3[i]) )
